<a href="https://colab.research.google.com/github/jerpint/jerpint.github.io/blob/master/colabs/Conway's_QR_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation
from scipy import signal
from PIL import Image, ImageOps
import urllib.request

%matplotlib inline

class GameOfLife:

    def __init__(self):
        '''Initialize the game.
        dim: dimensions of the board
        p: probability of a cell being dead at init
        seed: (optional) for reproducibility, set to None for a new random state
        init_state: (optional) a np.array grid to start the game with
        '''
        self.kernel = [
          [1, 1, 1],
          [1, 0, 1],
          [1, 1, 1],
        ]

        self.kernel = np.ones((3, 3))
        self.kernel[1, 1] = 0

    def set_random_state(self, dim=(100, 100), p=0.9, seed=None):
      if seed:
          np.random.seed(seed)
      self.state = (np.random.random(dim) > p).astype('int')

    def set_empty_state(self, dim=(100, 100)):
      self.state = np.zeros(dim)
      
    def set_state_from_array(self, array):
      self.state = array.copy()

    def count_neighbors(self):
        '''
        Count the number of live neighbors each cell in self.state has with convolutions.
        '''
        self.neighbors = (
            signal.convolve2d(self.state, 
                              self.kernel,
                              boundary='fill',
                              fillvalue=0,
                              mode='same').astype('int')
        )
        
    def place_blob(self, blob, i, j):
        '''Place a blob at coordinates i,j
        blob: ndarray of zeros and ones
        i: int
        j: int
        '''
        try:
            self.state[i:i+blob.shape[0], j:j+blob.shape[1]] = blob
        except:
            print("Check bounds of box vs size of game!")

    def step(self):
        '''Update the game based on conway game of life rules'''
        # Count the number of neighbors via convolution
        self.count_neighbors()

        # Copy of initial state
        self.new_state = self.state

        # Rebirth if cell is dead and has three live neighbors
        self.new_state += np.logical_and(self.neighbors == 3, self.state == 0)

        # Death if cell has less than 2 neighbors
        self.new_state -= np.logical_and(self.neighbors < 2, self.state == 1)

        # Death if cell has more than 3 neighbors
        self.new_state -= np.logical_and(self.neighbors > 3, self.state == 1)

        # Update game state
        self.state = self.new_state

    def animate_game(self, n_steps=100):

        def init():
            im.set_data(self.state)
            return [im]

        # animation function.  This is called sequentially
        def animate(i):
            im.set_array(self.state)
            if i < 9:
              # buffer
              pass
            else:
              self.step()
            return [im]

        # prepare the figures
        fig, ax = plt.subplots(figsize=(8, 8))
        plt.axis('off')
        im = plt.imshow(self.state, cmap='gray')

        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, animate, init_func=init,
                                      frames=n_steps, interval=120, blit=True)

        plt.close()

        return HTML(anim.to_html5_video())


def URL_to_pil_img(URL):
  """Fetch an image from a URL and load it as a PIL Image."""
  urllib.request.urlretrieve(URL, "qrcode.png")
  pil_img = Image.open("qrcode.png") # .resize((new_w, new_h), Image.ANTIALIAS)
  return pil_img

def binarize_qr_code(pil_img):
  """Convert a qr code to a binary map + resize it."""
  # resize
  old_w, old_h = pil_img.size
  new_w = 400
  new_h = int(new_w / old_w * old_h)
  pil_img = pil_img.resize((new_w, new_h), Image.ANTIALIAS)

  # convert to grayscale
  gray_image = np.array(ImageOps.grayscale(pil_img))

  # convert to binary map
  game_array = (gray_image > np.mean(gray_image)).astype(int)

  return game_array

In [2]:
# get QR Code
game_array = binarize_qr_code(URL_to_pil_img(URL="https://i.stack.imgur.com/yG7Os.png")) # load a QR code as a binary map

# instantiate game
gol = GameOfLife() 
gol.set_state_from_array(array=game_array) # set the qr code as the initial state

# play and view the game
display((gol.animate_game(n_steps=50))) 